## 模型构建

**模型构建两要素**

- 样例
    
        class Net(nn.Module):
            ##初始化构造模型
            def __init__(self, classes):
                super(LeNet, self).__init__()
                self.conv1 = nn.Conv2d(3, 6, 5)
                ....
            
            ##前向运算
            def forward(self, x):
                out = F.relu(self.conv1(x))
                out = F.max_pool2d(out, 2)
                out = F.relu(self.conv2(out))
                
                .....
                
                return out
            
            ##初始化模型
            def initialize_weights(self):
                for m in self.modules():
                    if isinstance(m, nn.Conv2d):
                        nn.init.xavier_normal_(m.weight.data)
                        if m.bias is not None:
                            m.bias.data.zero_()
                    ...




- 构建子模块：

       def __init__()函数

- 拼接子模块

       def forward()函数

**pytorch的神经网络模块**

torch.nn模块

- nn.parameter

    张量子类，表示可学习参数

- nn.module

    所有网络层基类

- nn.functional

    函数具体实现，如卷积，池化，激活函数

- nn.init

    基本初始化方法

![](./img/module.png)

**nn.module**

- 一个module可以包含多个module

- 一个module相当于运算，必须实现forward()函数

- 每个module都有8个字典管理属性



In [1]:
'''
模型创建实例
'''
import torch.nn as nn
import torch.nn.functional as F


class LeNet(nn.Module):
    def __init__(self, classes):
        super(LeNet, self).__init__()
        self.conv1 = nn.Conv2d(3, 6, 5)
        self.conv2 = nn.Conv2d(6, 16, 5)
        self.fc1 = nn.Linear(16 * 5 * 5, 120)
        self.fc2 = nn.Linear(120, 84)
        self.fc3 = nn.Linear(84, classes)

    def forward(self, x):
        out = F.relu(self.conv1(x))
        out = F.max_pool2d(out, 2)
        out = F.relu(self.conv2(out))
        out = F.max_pool2d(out, 2)
        out = out.view(out.size(0), -1)
        out = F.relu(self.fc1(out))
        out = F.relu(self.fc2(out))
        out = self.fc3(out)
        return out

    def initialize_weights(self):
        for m in self.modules():
            if isinstance(m, nn.Conv2d):
                nn.init.xavier_normal_(m.weight.data)
                if m.bias is not None:
                    m.bias.data.zero_()
            elif isinstance(m, nn.BatchNorm2d):
                m.weight.data.fill_(1)
                m.bias.data.zero_()
            elif isinstance(m, nn.Linear):
                nn.init.normal_(m.weight.data, 0, 0.1)
                m.bias.data.zero_()

## 模型容器

**Containers**

- nn.Sequential

    按顺序包装多个网络层
    
- nn.ModuleList

    像python的list一样包装多个网络层

- nn.ModuleDict

    像python的字典一样保证网络层

In [2]:
'''
nn.Sequential构建模型，能够自行反向传播

只能通过索引获取每个层
'''


class LeNet2(nn.Module):
    def __init__(self, classes):
        super(LeNet2, self).__init__()
        self.features = nn.Sequential(nn.Conv2d(3, 6, 5), nn.ReLU(),
                                      nn.MaxPool2d(2, 2), nn.Conv2d(6, 16, 5),
                                      nn.ReLU(), nn.MaxPool2d(2, 2))
        self.classifier = nn.Sequential(nn.Linear(16 * 5 * 5, 120), nn.ReLU(),
                                        nn.Linear(120, 84), nn.ReLU(),
                                        nn.Linear(84, classes))

    def forward(self, x):
        x = self.features(x)
        x = x.view(x.size()[0], -1)
        x = self.classifier(x)
        return x

In [3]:
'''
通过有序字典对构建网络模型并命名
'''


class LeNetSequentialOrderDict(nn.Module):
    def __init__(self, classes):
        super(LeNetSequentialOrderDict, self).__init__()

        self.features = nn.Sequential(
            OrderedDict({
                'conv1': nn.Conv2d(3, 6, 5),
                'relu1': nn.ReLU(inplace=True),
                'pool1': nn.MaxPool2d(kernel_size=2, stride=2),
                'conv2': nn.Conv2d(6, 16, 5),
                'relu2': nn.ReLU(inplace=True),
                'pool2': nn.MaxPool2d(kernel_size=2, stride=2),
            }))

        self.classifier = nn.Sequential(
            OrderedDict({
                'fc1': nn.Linear(16 * 5 * 5, 120),
                'relu3': nn.ReLU(),
                'fc2': nn.Linear(120, 84),
                'relu4': nn.ReLU(inplace=True),
                'fc3': nn.Linear(84, classes),
            }))

    def forward(self, x):
        x = self.features(x)
        x = x.view(x.size()[0], -1)
        x = self.classifier(x)
        return x

    nn.ModuleList以迭代方式调用网络层

- append()

    在ModuleList后面添加网络层

- extend()

    拼接两个ModuleList

- insert()

    指定ModuleList的某个位置插入网络层

In [4]:
class ModuleList(nn.Module):
    def __init__(self):
        super(ModuleList, self).__init__()
        self.linears = nn.ModuleList([nn.Linear(10, 10) for i in range(20)])

    def forward(self, x):
        for i, linear in enumerate(self.linears):
            x = linear(x)
        return x

    nn.ModuleDict可以通过索引方式构建网络层
    
- clear()

    清空nn.ModuleDict

- items

    返回键值对

- keys

    返回字典的键
    
- values

    返回键的值
    
- pop()

    返回键值对并删除

In [5]:
class ModuleDict(nn.Module):
    def __init__(self):
        super(ModuleDict, self).__init__()
        self.choices = nn.ModuleDict({
            'conv': nn.Conv2d(10, 10, 3),
            'pool': nn.MaxPool2d(3)
        })

        self.activations = nn.ModuleDict({
            'relu': nn.ReLU(),
            'prelu': nn.PReLU()
        })

    def forward(self, x, choice, act):
        x = self.choices[choice](x)
        x = self.activations[act](x)
        return x


net = ModuleDict()

fake_img = torch.randn((4, 10, 32, 32))

output = net(fake_img, 'conv', 'relu')

print(output)

NameError: name 'torch' is not defined